In [0]:
import pandas as pd
import numpy as np

import logging
import os
import sys

!pip install simpletransformers

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [0]:
from simpletransformers.classification import ClassificationModel

In [0]:
from transformers import BertForSequenceClassification, BertConfig

In [62]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

INFO:filelock:Lock 139968663327072 acquired on /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.8f56353af4a709bf5ff0fbc915d8f5b42bfff892cbb6ac98c3c45f481a03c685.lock


INFO:filelock:Lock 139968663327072 released on /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.8f56353af4a709bf5ff0fbc915d8f5b42bfff892cbb6ac98c3c45f481a03c685.lock


INFO:filelock:Lock 139964166906488 acquired on /root/.cache/torch/transformers/aa1ef1aede4482d0dbcd4d52baad8ae300e60902e88fcb0bebdec09afd232066.36ca03ab34a1a5d5fa7bc3d03d55c4fa650fed07220e2eeebc06ce58d0e9a157.lock


INFO:filelock:Lock 139964166906488 released on /root/.cache/torch/transformers/aa1ef1aede4482d0dbcd4d52baad8ae300e60902e88fcb0bebdec09afd232066.36ca03ab34a1a5d5fa7bc3d03d55c4fa650fed07220e2eeebc06ce58d0e9a157.lock


In [0]:
model

In [0]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [0]:
df = pd.read_csv('Statin_Data.csv')

X = df.loc[:,df.columns!='Category']
y = df.Category

trainx, testx, trainy, testy = train_test_split(X,y, test_size = 0.2, random_state= 123)

trainDf = pd.concat([trainx,trainy], axis = 1)
testDf = pd.concat([testx,testy], axis = 1)

trainDf.to_csv("train.csv",index = False, index_label = False)
testDf.to_csv("test.csv",index = False, index_label = False)

In [0]:
labelList = ['u', 'h', 'o', 'i', 'm', 'x', 'y', 'z']

labelMap = {}
for i in range(len(labelList)):
  labelMap[labelList[i]] = i

invLabelMap = {k:v for v,k in labelMap.items()}

In [0]:
trainDf['labels'] = trainDf.Category.apply(lambda x : labelMap[x])
testDf['labels'] = testDf.Category.apply(lambda x : labelMap[x])

In [0]:
trainDf = trainDf[['text','labels']]
testDf = testDf[['text','labels']]

In [43]:
model = ClassificationModel('xlnet', 'xlnet-base-cased', num_labels=8, use_cuda=True,
                            args={'reprocess_input_data': True, 'overwrite_output_dir': True, 'fp16':False,
                                  'num_train_epochs':3}) 

INFO:filelock:Lock 139971352225216 acquired on /root/.cache/torch/transformers/c9cc6e53904f7f3679a31ec4af244f4419e25ebc8e71ebf8c558a31cbcf07fc8.8df552e150a401a37ae808caf2a2c86fb6fedaa1f6963d1f21fbf3d0085c9e74.lock


INFO:filelock:Lock 139971352225216 released on /root/.cache/torch/transformers/c9cc6e53904f7f3679a31ec4af244f4419e25ebc8e71ebf8c558a31cbcf07fc8.8df552e150a401a37ae808caf2a2c86fb6fedaa1f6963d1f21fbf3d0085c9e74.lock


INFO:filelock:Lock 139964161071592 acquired on /root/.cache/torch/transformers/24197ba0ce5dbfe23924431610704c88e2c0371afa49149360e4c823219ab474.7eac4fe898a021204e63c88c00ea68c60443c57f94b4bc3c02adbde6465745ac.lock


INFO:filelock:Lock 139964161071592 released on /root/.cache/torch/transformers/24197ba0ce5dbfe23924431610704c88e2c0371afa49149360e4c823219ab474.7eac4fe898a021204e63c88c00ea68c60443c57f94b4bc3c02adbde6465745ac.lock


INFO:filelock:Lock 139968516017512 acquired on /root/.cache/torch/transformers/dad589d582573df0293448af5109cb6981ca77239ed314e15ca63b7b8a318ddd.8b10bd978b5d01c21303cc761fc9ecd464419b3bf921864a355ba807cfbfafa8.lock


INFO:filelock:Lock 139968516017512 released on /root/.cache/torch/transformers/dad589d582573df0293448af5109cb6981ca77239ed314e15ca63b7b8a318ddd.8b10bd978b5d01c21303cc761fc9ecd464419b3bf921864a355ba807cfbfafa8.lock


In [44]:
 model.train_model(trainDf)

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1340: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Running loss: 1.544127

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:224: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Running loss: 3.448634


Running loss: 2.464886


Running loss: 0.000975



INFO:simpletransformers.classification.classification_model: Training of xlnet model complete. Saved to outputs/.


In [45]:
out = model.eval_model(testDf)

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1340: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
INFO:simpletransformers.classification.classification_model:{'mcc': 0.6866601807043742, 'eval_loss': 0.9055769318473827}


In [36]:
out[1].shape

(2531, 8)

In [0]:
preds = out[1]

y_pred = np.argmax(preds,1)

In [38]:
accuracy_score(testDf.labels.values, y_pred)

0.780323982615567

In [28]:
testDf.labels.values

array([4, 1, 5, ..., 4, 4, 4])

In [0]:
preds = out[1]

y_pred = np.argmax(preds,1)

In [47]:
accuracy_score(testDf.labels.values, y_pred)

0.7977084156459897

In [0]:
from sklearn.metrics import confusion_matrix

In [0]:
testDf['preds'] = y_pred

In [50]:
testDf.head()

,text,labels,preds
4272,@stuves what. australia: - celebrex $26 - cymb...,4,4
661,@missjupiter1957 @bluebarontrader pt. 4 zocor ...,1,1
12261,"I know that Kendrick will have it sown up, but...",5,5
3594,"agree with dr. omrani's answer to ""i take 50mg...",3,3
9853,I don't like @realDonaldTrump having Crestor d...,4,4


In [0]:
testDf['labels'] = testDf.labels.apply(lambda x : invLabelMap[x])
testDf['preds'] = testDf.preds.apply(lambda x : invLabelMap[x])

In [0]:
testDf.to_csv('XLNet_Preds.csv',index = False, index_label=False)

In [21]:
!git clone https://github.com/NVIDIA/apex
!cd apex
!pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./


Cloning into 'apex'...
remote: Enumerating objects: 6153, done.
remote: Total 6153 (delta 0), reused 0 (delta 0), pack-reused 6153
Receiving objects: 100% (6153/6153), 13.63 MiB | 26.73 MiB/s, done.
Resolving deltas: 100% (4042/4042), done.
/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/install.py:283: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-2siwqo06
Created temporary directory: /tmp/pip-req-tracker-v01y_k12
Created requirements tracker '/tmp/pip-req-tracker-v01y_k12'
Created temporary directory: /tmp/pip-install-g2l_jh7o
Cleaning up...
Removed build tracker '/tmp/pip-req-tracker-v01y_k12'
ERROR: Directory './' is not installable. Neither 'setup.py' nor 'pyproject.toml' found.
Exception information:
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/pip/_internal/

In [0]:
result, model_outputs, wrong_predictions = model.eval_model(eval_df)